In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-uvtjad9v
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-uvtjad9v
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4306 sha256=c0f1e1c09d63db02920d6aeec4f225e586315c9e99b302c15a1b4833c845c043
  Stored in directory: /tmp/pip-ephem-wheel-cache-ziub99_b/wheels/c5/2b/c0/87008e795a14bbcdfc7c846a00d06981916331eb980b6c8bdf
Successfully built NVCCPlugin


In [ ]:
 import torch
 print('__CUDA Device Name:',torch.cuda.get_device_name(0))

__CUDA Device Name: Tesla K80


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%writefile global.cu
//matrix multiplication simple approach
#include <stdio.h>
#include <stdlib.h>
#include <assert.h>

#define BLOCK_SIZE_X 16
#define BLOCK_SIZE_Y 16

__global__ void gpu_matrix_mult(int *a,int *b, int *c, int m, int n, int k)
{ 
    int row = blockIdx.y * blockDim.y + threadIdx.y; 
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int sum = 0;
    if( col < k && row < m)
    {
        for(int i = 0; i < n; i++) 
        {
            sum += a[row * n + i] * b[i * k + col];
        }
        c[row * k + col] = sum;
    }
}

void cpu_matrix_mult(int *h_a, int *h_b, int *h_result, int m, int n, int k) {
    for (int i = 0; i < m; ++i) 
    {
        for (int j = 0; j < k; ++j) 
        {
            int tmp = 0.0;
            for (int h = 0; h < n; ++h) 
            {
                tmp += h_a[i * n + h] * h_b[h * k + j];
            }
            h_result[i * k + j] = tmp;
        }
    }
}

int main(int argc, char const *argv[])
{

  
    int m=8192, n=m, k=m;
 
    srand(3333);

    // allocate memory in host RAM, h_cc is used to store CPU result
    int *h_a, *h_b, *h_c, *h_cc;
    h_a = (int *)malloc((sizeof(int)*m*n)) ;
    h_b = (int *)malloc((sizeof(int)*m*n)) ;
    h_c = (int *)malloc((sizeof(int)*m*n)) ;
    h_cc = (int *)malloc((sizeof(int)*m*n)) ;
    float gpu_elapsed_time_ms, cpu_elapsed_time_ms;
     // random initialize matrix A
    for (int i = 0; i < m; ++i) {
        for (int j = 0; j < n; ++j) {
            h_a[i * n + j] =random()%4;
        }
    }

    // random initialize matrix B
    for (int i = 0; i < n; ++i) {
        for (int j = 0; j < k; ++j) {
            h_b[i * k + j] = random()%4;
        }
    }

    // some events to count the execution time
    cudaEvent_t start, stop;
    cudaEventCreate(&stop);
cudaEventCreate(&start);
    // start to count execution time of GPU version
    // Allocate memory space on the device 
    int *d_a, *d_b, *d_c;
    cudaMalloc(&d_a, sizeof(int)*m*n);
    cudaMalloc(&d_b, sizeof(int)*n*k);
    cudaMalloc(&d_c, sizeof(int)*m*k);

    // copy matrix A and B from host to device memory
    cudaMemcpy(d_a, h_a, sizeof(int)*m*n, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, sizeof(int)*n*k, cudaMemcpyHostToDevice);

    unsigned int grid_rows = m/BLOCK_SIZE_Y;
    unsigned int grid_cols = k/BLOCK_SIZE_X;
    dim3 dimGrid(grid_cols, grid_rows);
    dim3 dimBlock(BLOCK_SIZE_X, BLOCK_SIZE_Y);
   
    // Launch kernel 
   
    cudaEventRecord(start, 0);
    gpu_matrix_mult<<<dimGrid, dimBlock>>>(d_a, d_b, d_c, m, n, k);    
 cudaEventRecord(stop,0);
 cudaEventSynchronize(stop);

    // compute time elapse on GPU computing
    cudaEventElapsedTime(&gpu_elapsed_time_ms, start, stop);
    printf("Time elapsed on matrix multiplication of %dx%d . %dx%d on GPU: %f ms.\n\n", m, n, n, k, gpu_elapsed_time_ms);
cudaMemcpy(h_c, d_c, sizeof(int)*m*k, cudaMemcpyDeviceToHost);
    // start the CPU version 
    cudaEventRecord(start, 0);
    //cpu_matrix_mult(h_a, h_b, h_cc, m, n, k);

    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&cpu_elapsed_time_ms, start, stop);
    printf("Time elapsed on matrix multiplication of %dx%d . %dx%d on CPU: %f ms.\n\n", m, n, n, k, cpu_elapsed_time_ms);


    // free memory
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    free(h_a);
    free(h_b);
    free(h_c);
    free(h_cc);
    return 0;
}

Writing global.cu


In [ ]:
!nvidia-smi

Sun Nov 21 15:41:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!nvcc -o global global.cu

In [ ]:
!nvprof ./global

==162== NVPROF is profiling process 162, command: ./global
==162== Warning: Auto boost enabled on device 0. Profiling results may be inconsistent.
Time elapsed on matrix multiplication of 8192x8192 . 8192x8192 on GPU: 0.003872 ms.

Time elapsed on matrix multiplication of 8192x8192 . 8192x8192 on CPU: 0.003872 ms.

==162== Profiling application: ./global
==162== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   69.18%  174.28ms         1  174.28ms  174.28ms  174.28ms  [CUDA memcpy DtoH]
                   30.82%  77.633ms         2  38.816ms  38.518ms  39.115ms  [CUDA memcpy HtoD]
      API calls:   45.29%  269.39ms         2  134.69ms  1.6600us  269.38ms  cudaEventCreate
                   42.62%  253.48ms         3  84.492ms  38.672ms  175.63ms  cudaMemcpy
                   11.54%  68.642ms         3  22.881ms  485.48us  34.114ms  cudaFree
                    0.28%  1.6389ms         3  546.30us  506.46us  591.92us 

In [ ]:
%%writefile shared.cu
//matrix multiplication w/ shared memory
#include <stdio.h>
#include <stdlib.h>
#include <assert.h>

#define BLOCK_SIZE 16 // 32x32
#define TILE_WIDTH BLOCK_SIZE

__global__ void gpu_matrix_mult(int *a,int *b, int *c, int m, int n, int k){ 
    
    __shared__ int A[TILE_WIDTH][TILE_WIDTH];
    __shared__ int B[TILE_WIDTH][TILE_WIDTH];

    int bx = blockIdx.x;
    int by = blockIdx.y;
    int tx = threadIdx.x;
    int ty = threadIdx.y;
    int row = by * blockDim.y + ty;
    int col = bx * blockDim.x + tx;

    int Csub = 0;

    for (int t=0; t<n/TILE_WIDTH; t++)
    {
      A[ty][tx] = a[row*n + t*TILE_WIDTH + tx];
      B[ty][tx] = b[(t*TILE_WIDTH + ty)*k + col];
      
      __syncthreads();
      for (int i=0; i<TILE_WIDTH; i++)
      {
          Csub += A[ty][i] * B[i][tx];
      }
      __syncthreads();
    }
    c[row*k + col] = Csub;
}


void cpu_matrix_mult(int *h_a, int *h_b, int *h_result, int m, int n, int k) {
    for (int i = 0; i < m; ++i) 
    {
        for (int j = 0; j < k; ++j) 
        {
            int tmp = 0.0;
            for (int h = 0; h < n; ++h) 
            {
                tmp += h_a[i * n + h] * h_b[h * k + j];
            }
            h_result[i * k + j] = tmp;
        }
    }
}


int main(int argc, char const *argv[])
{
    int m=8192, n=m, k=m;
 
    srand(3333);

    // allocate memory in host RAM, h_cc is used to store CPU result
    int *h_a, *h_b, *h_c, *h_cc;
    h_a = (int *)malloc((sizeof(int)*m*n)) ;
    h_b = (int *)malloc((sizeof(int)*m*n)) ;
    h_c = (int *)malloc((sizeof(int)*m*n)) ;
    h_cc = (int *)malloc((sizeof(int)*m*n)) ;
    float gpu_elapsed_time_ms, cpu_elapsed_time_ms;

    // some events to count the execution time
    cudaEvent_t start, stop;
    cudaEventCreate(&stop);
cudaEventCreate(&start);
    // start to count execution time of GPU version
    // Allocate memory space on the device 
    int *d_a, *d_b, *d_c;
    cudaMalloc(&d_a, sizeof(int)*m*n);
    cudaMalloc(&d_b, sizeof(int)*n*k);
    cudaMalloc(&d_c, sizeof(int)*m*k);

    // copy matrix A and B from host to device memory
    cudaMemcpy(d_a, h_a, sizeof(int)*m*n, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, sizeof(int)*n*k, cudaMemcpyHostToDevice);

    unsigned int grid_rows = m/BLOCK_SIZE;
    unsigned int grid_cols = k/BLOCK_SIZE;
    dim3 dimGrid(grid_cols, grid_rows);
    dim3 dimBlock(BLOCK_SIZE, BLOCK_SIZE);
    
    // Launch kernel 
   
    cudaEventRecord(start, 0);
    gpu_matrix_mult<<<dimGrid, dimBlock>>>(d_a, d_b, d_c, m, n, k);    

    cudaEventRecord(stop,0);
    cudaEventSynchronize(stop);

    // compute time elapse on GPU computing
    cudaEventElapsedTime(&gpu_elapsed_time_ms, start, stop);
    printf("Time elapsed on matrix multiplication of %dx%d . %dx%d on GPU: %f ms.\n\n", m, n, n, k, gpu_elapsed_time_ms);
cudaMemcpy(h_c, d_c, sizeof(int)*m*k, cudaMemcpyDeviceToHost);
    // start the CPU version 
    cudaEventRecord(start, 0);
    //cpu_matrix_mult(h_a, h_b, h_cc, m, n, k);

    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&cpu_elapsed_time_ms, start, stop);
    printf("Time elapsed on matrix multiplication of %dx%d . %dx%d on CPU: %f ms.\n\n", m, n, n, k, cpu_elapsed_time_ms);

    // free memory
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    free(h_a);
    free(h_b);
    free(h_c);
    free(h_cc);
    return 0;
}

Overwriting shared.cu


In [ ]:
!nvcc -o shared shared.cu

In [ ]:
!./shared

Time elapsed on matrix multiplication of 8192x8192 . 8192x8192 on GPU: 0.003680 ms.

Time elapsed on matrix multiplication of 8192x8192 . 8192x8192 on CPU: 0.003904 ms.



In [ ]:
!/usr/local/cuda-10.0/NsightCompute-1.0/nv-nsight-cu-cli ./shared

==PROF== Profiling -    1: 0%....50%....100%
Time elapsed on matrix multiplication of 1024x1024 . 1024x1024 on GPU: 4336.388184 ms.

Time elapsed on matrix multiplication of 1024x1024 . 1024x1024 on CPU: 0.003968 ms.

==PROF== Report: profile.nsight-cuprof-report
gpu_matrix_mult, 2021-Aug-11 15:34:18
Section: GPU Speed Of Light
---------------------------------------------------------------------- --------------- ------------------------------
Memory Frequency                                                         cycle/nsecond                           1.25
SOL FB                                                                               %                           5.21
Elapsed Cycles                                                                   cycle                   5,231,258.60
SM Frequency                                                             cycle/usecond                         583.60
Memory [%]                                                                      

In [ ]:
%%cu
//matrix multiplication w/ shared memory new
#include <stdio.h>
#include <stdlib.h>
#include <assert.h>

#define BLOCK_SIZE 32 // 32x32
#define TILE_WIDTH 64

__global__ void gpu_matrix_mult(int *a,int *b, int *c, int m, int n, int k){ 
    
    __shared__ int A[TILE_WIDTH][TILE_WIDTH];
    __shared__ int B[TILE_WIDTH][TILE_WIDTH];

    int bx = blockIdx.x;
    int by = blockIdx.y;
    int tx = threadIdx.x;
    int ty = threadIdx.y;
    int row = by * blockDim.y + ty;
    int col = bx * blockDim.x + tx;

    int Csub0 = 0;
    int Csub1 = 0;
    int Csub2 = 0;
    int Csub3 = 0;

    for (int t=0; t<n/TILE_WIDTH; t++)
    {
      A[tx*2][ty*2] = a[2*row*n + t*TILE_WIDTH + 2*tx];
      A[tx*2+1][ty*2] = a[2*row*n + t*TILE_WIDTH + 2*tx+1];
      A[tx*2][ty*2+1] = a[2*row*n + t*TILE_WIDTH + 2*tx+n];
      A[tx*2+1][ty*2+1] = a[2*row*n + t*TILE_WIDTH + 2*tx+n+1];
  
      B[tx*2][ty*2] = b[(t*TILE_WIDTH + 2*ty)*k + 2*col];
      B[tx*2+1][ty*2] = b[(t*TILE_WIDTH + 2*ty)*k + 2*col+1];
      B[tx*2][ty*2+1] = b[(t*TILE_WIDTH + 2*ty)*k + 2*col+n];
      B[tx*2+1][ty*2+1] = b[(t*TILE_WIDTH + 2*ty)*k + 2*col+n+1];
     
    /*  B[tx*2][ty*2] = b[2*row*n + t*TILE_WIDTH + 2*tx];
      B[tx*2+1][ty*2] = b[2*row*n + t*TILE_WIDTH + 2*tx+1];
      B[tx*2][ty*2+1] = b[2*row*n + t*TILE_WIDTH + 2*tx+n];
      B[tx*2+1][ty*2+1] = b[2*row*n + t*TILE_WIDTH + 2*tx+n+1];
     */
      __syncthreads();
      
      for (int i=0; i<TILE_WIDTH; i++)
      {
          Csub0 += A[i][2*ty] * B[2*tx][i];
       //Csub0 += A[i][2*ty] * B[i][2*ty];
      } 
      for (int i=0; i<TILE_WIDTH; i++)
      {
          Csub1 += A[i][2*ty] * B[2*tx+1][i];
       //Csub1 += A[i][2*ty] * B[i][2*ty];
      }
      for (int i=0; i<TILE_WIDTH; i++)
      {
          Csub2 += A[i][2*ty+1] * B[2*tx][i];
       //Csub2 += A[i][2*ty+1] * B[i][2*ty+1];
      }
      for (int i=0; i<TILE_WIDTH; i++)
      {
          
          Csub3 += A[i][2*ty+1] * B[2*tx+1][i];
       //Csub3 += A[i][2*ty+1] * B[i][2*ty+1];
      }
      __syncthreads();
    }
    c[by*TILE_WIDTH*n+2*ty*n+bx*TILE_WIDTH+2*tx] = Csub0;
    c[by*TILE_WIDTH*n+2*ty*n+bx*TILE_WIDTH+2*tx+1] = Csub1;
    c[by*TILE_WIDTH*n+2*ty*n+bx*TILE_WIDTH+2*tx+n] = Csub2;
    c[by*TILE_WIDTH*n+2*ty*n+bx*TILE_WIDTH+2*tx+n+1] = Csub3; 
} 

int main(int argc, char const *argv[])
{
    int m=512*16, n=m, k=m;
 
    srand(3333);

    // allocate memory in host RAM, h_cc is used to store CPU result
    int *h_a, *h_b, *h_c, *h_cc;
    h_a = (int *)malloc((sizeof(int)*m*n)) ;
    h_b = (int *)malloc((sizeof(int)*m*n)) ;
    h_c = (int *)malloc((sizeof(int)*m*n)) ;
    h_cc = (int *)malloc((sizeof(int)*m*n)) ;
    float gpu_elapsed_time_ms, cpu_elapsed_time_ms;

    // some events to count the execution time
    cudaEvent_t start, stop;
    cudaEventCreate(&stop);
cudaEventCreate(&start);
    // start to count execution time of GPU version
    // Allocate memory space on the device 
    int *d_a, *d_b, *d_c;
    cudaMalloc(&d_a, sizeof(int)*m*n);
    cudaMalloc(&d_b, sizeof(int)*n*k);
    cudaMalloc(&d_c, sizeof(int)*m*k);

    // copy matrix A and B from host to device memory
    cudaMemcpy(d_a, h_a, sizeof(int)*m*n, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, sizeof(int)*n*k, cudaMemcpyHostToDevice);

    unsigned int grid_rows = m/BLOCK_SIZE/2;
    unsigned int grid_cols = k/BLOCK_SIZE/2;
    dim3 dimGrid(grid_cols, grid_rows);
    dim3 dimBlock(BLOCK_SIZE, BLOCK_SIZE);
    
    // Launch kernel 
   
    cudaEventRecord(start, 0);
    gpu_matrix_mult<<<dimGrid, dimBlock>>>(d_a, d_b, d_c, m, n, k);    

    cudaEventRecord(stop,0);
    cudaEventSynchronize(stop);

    // compute time elapse on GPU computing
    cudaEventElapsedTime(&gpu_elapsed_time_ms, start, stop);
    printf("Time elapsed on matrix multiplication of %dx%d . %dx%d on GPU: %f ms.\n\n", m, n, n, k, gpu_elapsed_time_ms);
cudaMemcpy(h_c, d_c, sizeof(int)*m*k, cudaMemcpyDeviceToHost);
    // start the CPU version 
    cudaEventRecord(start, 0);
    //cpu_matrix_mult(h_a, h_b, h_cc, m, n, k);

    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&cpu_elapsed_time_ms, start, stop);
    printf("Time elapsed on matrix multiplication of %dx%d . %dx%d on CPU: %f ms.\n\n", m, n, n, k, cpu_elapsed_time_ms);


    // free memory
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    free(h_a);
    free(h_b);
    free(h_c);
    free(h_cc);
    return 0;
}

Time elapsed on matrix multiplication of 8192x8192 . 8192x8192 on GPU: 5439.753906 ms.

Time elapsed on matrix multiplication of 8192x8192 . 8192x8192 on CPU: 0.002720 ms.




In [ ]:
%%cu
// test fermi
#include <stdio.h>
#include <stdlib.h>
#include <assert.h>
#define size 4
#define BLOCK_SIZE 16
#define TILE_WIDTH 64
#define TILE_DIM 32
#define BLOCK_ROWS 8
__global__ void gpu_matrix_mult3(int *a,int *b, int *c, int m, int n, int k){ 
    
    __shared__ int A[TILE_WIDTH][TILE_WIDTH];
    __shared__ int B[TILE_WIDTH][TILE_WIDTH];

    int bx = blockIdx.x;
    int by = blockIdx.y;
    int tx = threadIdx.x;
    int ty = threadIdx.y;
    int row = by * blockDim.y + ty;
    int col = bx * blockDim.x + tx;

    int sum[size*size]={0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0};
    for (int t=0; t<n/TILE_WIDTH; t++)
    {
      for (int j=0; j<size; j++){
          for (int i=0; i<size; i++){
              A[ty+j*BLOCK_SIZE][tx+i*BLOCK_SIZE] = a[by*TILE_WIDTH*n + t*TILE_WIDTH+ty*n+BLOCK_SIZE*n*j+BLOCK_SIZE*i+tx];
              B[ty+j*BLOCK_SIZE][tx+i*BLOCK_SIZE] = b[t*TILE_WIDTH*n + bx*TILE_WIDTH+ty*n+BLOCK_SIZE*n*j+BLOCK_SIZE*i+tx];
          }
      }
      __syncthreads();
      for (int i=0; i<TILE_WIDTH; i++){
          int b_t[size];
          int a_t[size];
          for (int k=0; k<size; k++){
              b_t[k]=B[i][size*tx+k];
              a_t[k]=A[size*ty+k][i];
          }
          for (int q=0; q<size; q++){
              for (int w=0; w<size; w++){
                  sum[size*q+w]+=b_t[w]*a_t[q];
                  printf("%i ", sum[size*q+w]);
              }
          }
      }
      __syncthreads();
    }
    for (int j=0; j<size; j++){
        for (int i=0; i<size; i++){
            c[by*TILE_WIDTH*n+size*ty*n+bx*TILE_WIDTH+size*tx+i+n*j]=sum[size*j+i];
            //c[by*TILE_WIDTH*n+bx*TILE_WIDTH+j*size*n+ty*n+tx]=sum[size*j+i];
        }
    }
}
__global__ void gpu_matrix_mult(int *a,int *b, int *c, int m, int n, int k){ 
    
    __shared__ int A[TILE_WIDTH][TILE_WIDTH];
    __shared__ int B[TILE_WIDTH][TILE_WIDTH];

    int bx = blockIdx.x;
    int by = blockIdx.y;
    int tx = threadIdx.x;
    int ty = threadIdx.y;
    int row = by * blockDim.y + ty;
    int col = bx * blockDim.x + tx;

    int Csub0 = 0;
    int Csub1 = 0;
    int Csub2 = 0;
    int Csub3 = 0;
    //int sum[4]={0,0,0,0};
    for (int t=0; t<n/TILE_WIDTH; t++)
    {
      A[ty][tx] = a[by*TILE_WIDTH*n + t*TILE_WIDTH + ty*n+tx];
      A[ty][tx+BLOCK_SIZE] = a[by*TILE_WIDTH*n + t*TILE_WIDTH + ty*n+tx+BLOCK_SIZE];
      A[ty+BLOCK_SIZE][tx] = a[by*TILE_WIDTH*n + t*TILE_WIDTH + ty*n+tx+BLOCK_SIZE*n];
      A[ty+BLOCK_SIZE][tx+BLOCK_SIZE] = a[by*TILE_WIDTH*n + t*TILE_WIDTH + ty*n+tx+BLOCK_SIZE*n+BLOCK_SIZE];
  
      /*B[ty*2][tx*2] = b[(t*TILE_WIDTH + 2*ty)*k + 2*col];
      B[ty*2][tx*2+1] = b[(t*TILE_WIDTH + 2*ty)*k + 2*col+1];
      B[ty*2+1][tx*2] = b[(t*TILE_WIDTH + 2*ty)*k + 2*col+n];
      B[ty*2+1][tx*2+1] = b[(t*TILE_WIDTH + 2*ty)*k + 2*col+n+1];*/
     
      B[ty][tx] = b[t*TILE_WIDTH*n + bx*TILE_WIDTH + ty*n + tx];
      B[ty][tx+BLOCK_SIZE] = b[t*TILE_WIDTH*n + bx*TILE_WIDTH + ty*n + tx + BLOCK_SIZE];
      B[ty+BLOCK_SIZE][tx] = b[t*TILE_WIDTH*n + bx*TILE_WIDTH + ty*n + tx + BLOCK_SIZE*n];
      B[ty+BLOCK_SIZE][tx+BLOCK_SIZE] = b[t*TILE_WIDTH*n + bx*TILE_WIDTH + ty*n + tx + BLOCK_SIZE*n + BLOCK_SIZE];

      __syncthreads();
      
      /*for (int i=0; i<TILE_WIDTH; i++){
          int b1 = B[i][2*tx];
          int b2 = B[i][2*tx+1];
          int a1 = A[2*ty][i];
          int a2 = A[2*ty+1][i];
          sum[0]+=b1*a1;
          sum[1]+=b2*a1;
          sum[2]+=a2*b1;
          sum[3]+=a2*b2;
      }*/

      for (int i=0; i<TILE_WIDTH; i++)
      {
          Csub0 += A[ty][i] * B[i][tx];       
      }
      for (int i=0; i<TILE_WIDTH; i++)
      {
          Csub1 += A[ty][i] * B[i][tx+BLOCK_SIZE];
      }
      for (int i=0; i<TILE_WIDTH; i++)
      {
          Csub2 += A[ty+BLOCK_SIZE][i] * B[i][tx];
      }
      for (int i=0; i<TILE_WIDTH; i++)
      {
          Csub3 += A[ty+BLOCK_SIZE][i] * B[i][tx+BLOCK_SIZE];
      }
      __syncthreads();
    }
    c[by*TILE_WIDTH*n+ty*n+bx*TILE_WIDTH+tx] = Csub0;
    c[by*TILE_WIDTH*n+ty*n+bx*TILE_WIDTH+tx+BLOCK_SIZE] = Csub1;
    c[by*TILE_WIDTH*n+ty*n+bx*TILE_WIDTH+tx+BLOCK_SIZE*n] = Csub2;
    c[by*TILE_WIDTH*n+ty*n+bx*TILE_WIDTH+tx+BLOCK_SIZE*n+BLOCK_SIZE] = Csub3;

    /*c[by*TILE_WIDTH*n+2*ty*n+bx*TILE_WIDTH+2*tx] = sum[0];
    c[by*TILE_WIDTH*n+2*ty*n+bx*TILE_WIDTH+2*tx+1] = sum[1];
    c[by*TILE_WIDTH*n+2*ty*n+bx*TILE_WIDTH+2*tx+n] = sum[2];
    c[by*TILE_WIDTH*n+2*ty*n+bx*TILE_WIDTH+2*tx+n+1] = sum[3];*/
} 

__global__ void gpu_matrix_mult2(int *a,int *b, int *c, int m, int n, int k){ 
    
    __shared__ int A[TILE_WIDTH][TILE_WIDTH];
    __shared__ int B[TILE_WIDTH][TILE_WIDTH];

    int bx = blockIdx.x;
    int by = blockIdx.y;
    int tx = threadIdx.x;
    int ty = threadIdx.y;
    int row = by * blockDim.y + ty;
    int col = bx * blockDim.x + tx;
    int Csub = 0;
    for (int t=0; t<n/TILE_WIDTH; t++)
    {
      A[ty][tx] = b[(t*TILE_WIDTH + ty)*k + col];//a[row*n + t*TILE_WIDTH + tx];
      B[ty][tx] = b[(t*TILE_WIDTH + ty)*k + col];//b[row*n + t*TILE_WIDTH + tx];
      
      __syncthreads();
      for (int i=0; i<TILE_WIDTH; i++)
      {
          Csub += A[ty][i] * B[ty][i]; 
      }
      __syncthreads();
    }
    c[row*k + col] = Csub;
}

void cpu_matrix_mult(int *h_a, int *h_b, int *h_result, int m, int n, int k) {
    for (int i = 0; i < m; ++i) 
    {
        for (int j = 0; j < k; ++j) 
        {
            int tmp = 0.0;
            for (int h = 0; h < n; ++h) 
            {
                tmp += h_a[i * n + h] * h_b[h * k + j];
            }
            h_result[i * k + j] = tmp;
        }
    }
}


int main(int argc, char const *argv[])
{
    int m=64;
    int n=m;
    int k=m;
 
    srand(3333);

    // allocate memory in host RAM, h_cc is used to store CPU result
    int *h_a, *h_b, *h_c, *h_cc;
    h_a = (int *)malloc((sizeof(int)*m*n)) ;
    h_b = (int *)malloc((sizeof(int)*m*n)) ;
    h_c = (int *)malloc((sizeof(int)*m*n)) ;
    h_cc = (int *)malloc((sizeof(int)*m*n)) ;

    // random initialize matrix A
    for (int i = 0; i < m; ++i) {
        for (int j = 0; j < n; ++j) {
            h_a[i * n + j] = random()%10;
        }
    }

    // random initialize matrix B
    for (int i = 0; i < n; ++i) {
        for (int j = 0; j < k; ++j) {
            h_b[i * k + j] = random()%10;
        }
    }

    float gpu_elapsed_time_ms, cpu_elapsed_time_ms;

    // some events to count the execution time
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // start to count execution time of GPU version
    // Allocate memory space on the device 
    int *d_a, *d_b, *d_c, *d_bt;
    cudaMalloc(&d_a, sizeof(int)*m*n);
    cudaMalloc(&d_b, sizeof(int)*n*k);
    cudaMalloc(&d_c, sizeof(int)*m*k);
    cudaMalloc(&d_bt, sizeof(int)*m*k);

    // copy matrix A and B from host to device memory
    cudaMemcpy(d_a, h_a, sizeof(int)*m*n, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, sizeof(int)*n*k, cudaMemcpyHostToDevice);

    unsigned int grid_rows = m/BLOCK_SIZE/4;
    unsigned int grid_cols = k/BLOCK_SIZE/4;
    dim3 dimGrid(grid_cols, grid_rows);

    dim3 dimGrid2(grid_cols, grid_rows);
    dim3 dimBlock2(BLOCK_SIZE, BLOCK_SIZE); 
 
    cudaEventRecord(start, 0);
    gpu_matrix_mult3<<<dimGrid, dimBlock2>>>(d_a, d_b, d_c, m, n, k);
    cudaDeviceSynchronize();
    cudaEventRecord(stop, 0);
 

    cudaEventElapsedTime(&gpu_elapsed_time_ms, start, stop);
    // Transefer results from device to host
    cudaMemcpy(h_c, d_c, sizeof(int)*m*k, cudaMemcpyDeviceToHost);
    cudaEventSynchronize(stop);
    cudaDeviceSynchronize();

    // compute time elapse on GPU computing
    printf("Time elapsed on matrix multiplication of %dx%d . %dx%d on GPU: %f ms.\n\n", m, n, n, k, gpu_elapsed_time_ms);

    // start the CPU version
    cudaEventRecord(start, 0);

    cpu_matrix_mult(h_a, h_b, h_cc, m, n, k);

    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&cpu_elapsed_time_ms, start, stop);
    printf("Time elapsed on matrix multiplication of %dx%d . %dx%d on CPU: %f ms.\n\n", m, n, n, k, cpu_elapsed_time_ms);

    // validate results computed by GPU
    int all_ok = 1;
    for (int i = 0; i < m; ++i)
    {
        for (int j = 0; j < k; ++j)
        {
            //printf("[%d][%d]:%d == [%d][%d]:%d, ", i, j, h_cc[i*k + j], i, j, h_c[i*k + j]);
         //printf("%d ",h_cc[i*k + j]);
            if(h_cc[i*k + j] != h_c[i*k + j])
            {
                all_ok = 0;
            }
        }
        //printf("\n");
    }

    // roughly compute speedup
    if(all_ok)
    {
        printf("all results are correct!!!, speedup = %f\n", cpu_elapsed_time_ms / gpu_elapsed_time_ms);
    }
    else
    {
        printf("incorrect results\n");
    }
    // free memory
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    free(h_a);
    free(h_b);
    free(h_c);
    free(h_cc);
    return 0;
}

0 20 36 0 12 16 20 16 32 24 16 12 20 8 24 4 0 45 81 0 27 36 45 36 72 54 36 27 45 18 54 9 0 10 18 0 6 8 10 8 16 12 8 6 10 4 12 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 20 36 0 12 16 20 16 32 24 16 12 20 8 24 4 0 5 9 0 3 4 5 4 8 6 4 3 5 2 6 1 0 15 27 0 9 12 15 12 24 18 12 9 15 6 18 3 0 30 54 0 18 24 30 24 48 36 24 18 30 12 36 6 0 5 9 0 3 4 5 4 8 6 4 3 5 2 6 1 0 40 72 0 24 32 40 32 64 48 32 24 40 16 48 8 0 25 45 0 15 20 25 20 40 30 20 15 25 10 30 5 0 35 63 0 21 28 35 28 56 42 28 21 35 14 42 7 0 35 63 0 21 28 35 28 56 42 28 21 35 14 42 7 0 10 18 0 6 8 10 8 16 12 8 6 10 4 12 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 15 27 0 9 12 15 12 24 18 12 9 15 6 18 3 4 20 12 24 0 28 28 28 36 4 20 20 20 36 4 28 9 45 27 54 0 63 63 63 81 9 45 45 45 81 9 63 4 20 12 24 0 28 28 28 36 4 20 20 20 36 4 28 1 5 3 6 0 7 7 7 9 1 5 5 5 9 1 7 3 15 9 18 0 21 21 21 27 3 15 15 15 27 3 21 6 30 18 36 0 42 42 42 54 6 30 30 30 54 6 42 2 10 6 12 0 14 14 14 18 2 10 10 10 18 2 14 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 5 3 6 0 7 7 7 9 1 5 5 5 

In [ ]:
%%cu
//matrix addition
#include <stdio.h>
#include <stdlib.h>
#include <assert.h>

#define BLOCK_SIZE_X 32
#define BLOCK_SIZE_Y 32
#define TILE_DIM 32
#define BLOCK_ROWS 8
__global__ void gpu_matrix_mult(int *a,int *b, int *c, int m, int n, int k)
{
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    c[row * k + col] = a[row * k + col]+b[row * k + col];
} 
__global__ void transposeNaive(int *odata, const int *idata)
{
  int x = blockIdx.x * TILE_DIM + threadIdx.x;
  int y = blockIdx.y * TILE_DIM + threadIdx.y;
  int width = gridDim.x * TILE_DIM;

  for (int j = 0; j < TILE_DIM; j+= BLOCK_ROWS)
    odata[x*width + (y+j)] = idata[(y+j)*width + x];
}

int main(int argc, char const *argv[])
{
    int m=8*1024, n=8*1024, k=8*1024;
 printf("a");
    srand(3333);

   // int *h_a, *h_b, *h_c, *h_cc;
   // h_a = (int *)malloc((sizeof(int)*m*n)) ;
   // h_b = (int *)malloc((sizeof(int)*m*n)) ;
   // h_c = (int *)malloc((sizeof(int)*m*n)) ;
    int h_a[m][n];
 int h_b[m][n];
 int h_c[m][n];
 //int h_cc[m][n];

    float gpu_elapsed_time_ms;

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    
    int *d_a, *d_b, *d_c;
    cudaMalloc(&d_a, sizeof(int)*m*n);
    cudaMalloc(&d_b, sizeof(int)*n*k);
    cudaMalloc(&d_c, sizeof(int)*m*k);

    cudaMemcpy(d_a, h_a, sizeof(int)*m*n, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, sizeof(int)*n*k, cudaMemcpyHostToDevice);
 
    unsigned int grid_cols = k/BLOCK_SIZE_X;
    unsigned int grid_rows = m/BLOCK_SIZE_Y;
    dim3 dimGrid(grid_cols, grid_rows);
    dim3 dimBlock(32, 32);
 
    cudaEventRecord(start, 0); 
    transposeNaive<<<dimGrid, dimBlock>>>(d_c, d_b);
    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&gpu_elapsed_time_ms, start, stop);
    printf("Time elapsed on matrix multiplication of %dx%d . %dx%d on GPU: %f ms.\n\n", m, n, n, k, gpu_elapsed_time_ms);

    cudaMemcpy(h_c, d_c, sizeof(int)*m*k, cudaMemcpyDeviceToHost);

    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    free(h_a);
    free(h_b);
    free(h_c);
    return 0;
}

In [ ]:
%%cu 
//sum of arrays values
#include <stdio.h>
#include <stdlib.h>
#include <assert.h>
#define size 64
#define BLOCK_SIZE 64

__global__ void gpu_matrix_mult(int *a,int *b, int n)
{
    int col = blockIdx.x * blockDim.x;
    int sum=0;
    for (int i=0; i<size; i++){ 
        sum+=a[(col+threadIdx.x)*size+i]; //sum+=a[col*size+i*threadIdx.x]; 
    }
    atomicAdd(b, sum);
} 

int main(int argc, char const *argv[])
{
    int n=128*1024*1024;
 
    int *h_a, *answer;
    h_a = (int *)malloc((sizeof(int)*n)) ;
    answer = (int *)malloc((sizeof(int)*1)) ;


    float gpu_elapsed_time_ms;

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    
    int *d_a, *answercuda;
    cudaMalloc(&d_a, sizeof(int)*n);
    cudaMalloc(&answercuda, sizeof(int)*1);

    cudaMemcpy(d_a, h_a, sizeof(int)*n, cudaMemcpyHostToDevice);
 

    unsigned int grid_cols = (n) / size / BLOCK_SIZE;
    dim3 dimGrid(grid_cols);
    dim3 dimBlock(BLOCK_SIZE);
 
    cudaEventRecord(start, 0); 
    gpu_matrix_mult<<<dimGrid, dimBlock>>>(d_a, answercuda, n);
    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&gpu_elapsed_time_ms, start, stop);
    printf("%f ms\n", gpu_elapsed_time_ms);

    cudaMemcpy(answer, answercuda, sizeof(int)*1, cudaMemcpyDeviceToHost);

    cudaFree(d_a);
    cudaFree(answercuda);
    free(h_a);
    free(answer);
    return 0;
}

11.649152 ms



In [ ]:
%%cu
//matrix multiplication w/ shared memory govno
#include <stdio.h>
#include <stdlib.h>
#include <assert.h>

#define BLOCK_SIZE 32 // 32x32
#define TILE_WIDTH 32

__global__ void gpu_matrix_mult(int *a,int *b, int *c, int m, int n, int k){ 
    
    __shared__ int A[110][110];
    __shared__ int B[111][111];

    int bx = blockIdx.x;
    int by = blockIdx.y;
    int tx = threadIdx.x;
    int ty = threadIdx.y;
    int row = by * blockDim.y + ty;
    int col = bx * blockDim.x + tx;

    int Csub = 0;

    for (int t=0; t<n/TILE_WIDTH; t++)
    {
      A[ty][tx] = a[row*n + t*TILE_WIDTH + tx];
      B[ty][tx] = b[(t*TILE_WIDTH + ty)*k + col];
      
      __syncthreads();
      for (int i=0; i<TILE_WIDTH; i++)
      {
          Csub += A[i][0];
      }
      __syncthreads();
    }
    c[row*k + col] = Csub;
}


void cpu_matrix_mult(int *h_a, int *h_b, int *h_result, int m, int n, int k) {
    for (int i = 0; i < m; ++i) 
    {
        for (int j = 0; j < k; ++j) 
        {
            int tmp = 0.0;
            for (int h = 0; h < n; ++h) 
            {
                tmp += h_a[i * n + h] * h_b[h * k + j];
            }
            h_result[i * k + j] = tmp;
        }
    }
}


int main(int argc, char const *argv[])
{

    int m=4*2048, n=m, k=m;
 
    srand(3333);

    // allocate memory in host RAM, h_cc is used to store CPU result
    int *h_a, *h_b, *h_c, *h_cc;
    h_a = (int *)malloc((sizeof(int)*m*n)) ;
    h_b = (int *)malloc((sizeof(int)*m*n)) ;
    h_c = (int *)malloc((sizeof(int)*m*n)) ;
    h_cc = (int *)malloc((sizeof(int)*m*n)) ;
    float gpu_elapsed_time_ms, cpu_elapsed_time_ms;

    // some events to count the execution time
    cudaEvent_t start, stop;
    cudaEventCreate(&stop);
cudaEventCreate(&start);
    // start to count execution time of GPU version
    // Allocate memory space on the device 
    int *d_a, *d_b, *d_c;
    cudaMalloc(&d_a, sizeof(int)*m*n);
    cudaMalloc(&d_b, sizeof(int)*n*k);
    cudaMalloc(&d_c, sizeof(int)*m*k);

    // copy matrix A and B from host to device memory
    cudaMemcpy(d_a, h_a, sizeof(int)*m*n, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, sizeof(int)*n*k, cudaMemcpyHostToDevice);

    unsigned int grid_rows = m/BLOCK_SIZE;
    unsigned int grid_cols = k/BLOCK_SIZE;
    dim3 dimGrid(grid_cols, grid_rows);
    dim3 dimBlock(BLOCK_SIZE, BLOCK_SIZE);
    
    // Launch kernel 
   
    cudaEventRecord(start, 0);
    gpu_matrix_mult<<<dimGrid, dimBlock>>>(d_a, d_b, d_c, m, n, k);    

    cudaEventRecord(stop,0);
    cudaEventSynchronize(stop);

    // compute time elapse on GPU computing
    cudaEventElapsedTime(&gpu_elapsed_time_ms, start, stop);
    printf("Time elapsed on matrix multiplication of %dx%d . %dx%d on GPU: %f ms.\n\n", m, n, n, k, gpu_elapsed_time_ms);
    cudaMemcpy(h_c, d_c, sizeof(int)*m*k, cudaMemcpyDeviceToHost);
    // start the CPU version 
    cudaEventRecord(start, 0);
    //cpu_matrix_mult(h_a, h_b, h_cc, m, n, k);

    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&cpu_elapsed_time_ms, start, stop);
    printf("Time elapsed on matrix multiplication of %dx%d . %dx%d on CPU: %f ms.\n\n", m, n, n, k, cpu_elapsed_time_ms);


    // free memory
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    free(h_a);
    free(h_b);
    free(h_c);
    free(h_cc);
    return 0;
}

Time elapsed on matrix multiplication of 8192x8192 . 8192x8192 on GPU: 810.394287 ms.

Time elapsed on matrix multiplication of 8192x8192 . 8192x8192 on CPU: 0.002688 ms.




In [ ]:
%%cu
// test
#include <stdio.h>
#include <stdlib.h>
#include <assert.h>

#define size 3
#define BLOCK_SIZE 26
#define TILE_WIDTH 78
__global__ void gpu_matrix_mult(int *a,int *b, int *c, int m, int n, int k){ 
    
    __shared__ int A[TILE_WIDTH][TILE_WIDTH];
    __shared__ int B[TILE_WIDTH][TILE_WIDTH];

    int bx = blockIdx.x;
    int by = blockIdx.y;
    int tx = threadIdx.x;
    int ty = threadIdx.y;
    int row = by * blockDim.y + ty;
    int col = bx * blockDim.x + tx;

    int sum[size*size];
    for (int t=0; t<n/TILE_WIDTH; t++)
    {
      for (int j=0; j<size; j++){
          for (int i=0; i<size; i++){
              A[ty+j*BLOCK_SIZE][tx+i*BLOCK_SIZE] = a[by*TILE_WIDTH*n + t*TILE_WIDTH+ty*n+BLOCK_SIZE*n*j+BLOCK_SIZE*i+tx];
              B[ty+j*BLOCK_SIZE][tx+i*BLOCK_SIZE] = b[t*TILE_WIDTH*n + bx*TILE_WIDTH+ty*n+BLOCK_SIZE*n*j+BLOCK_SIZE*i+tx];
          }
      }
      __syncthreads();
     
      for (int i=0; i<TILE_WIDTH; i++){
          int b_t[size];
          int a_t[size];
          for (int k=0; k<size; k++){
              b_t[k]=B[i][size*tx+k];
              a_t[k]=A[size*ty+k][i];
          }
          for (int q=0; q<size; q++){
              for (int w=0; w<size; w++){
                  sum[size*q+w]+=b_t[w]*a_t[q];
              }
          }
      }
      __syncthreads();
    }
    for (int j=0; j<size; j++){
        for (int i=0; i<size; i++){
            c[by*TILE_WIDTH*n+size*ty*n+bx*TILE_WIDTH+size*tx+i+n*j]=sum[size*j+i];
        }
    }
}


void cpu_matrix_mult(int *h_a, int *h_b, int *h_result, int m, int n, int k) {
    for (int i = 0; i < m; ++i) 
    {
        for (int j = 0; j < k; ++j) 
        {
            int tmp = 0.0;
            for (int h = 0; h < n; ++h) 
            {
                tmp += h_a[i * n + h] * h_b[h * k + j];
            }
            h_result[i * k + j] = tmp;
        }
    }
}


int main(int argc, char const *argv[])
{
    int m=8*1024;
    int n=m;
    int k=m;
 
    srand(3333);

    // allocate memory in host RAM, h_cc is used to store CPU result
    int *h_a, *h_b, *h_c, *h_cc;
    h_a = (int *)malloc((sizeof(int)*m*n)) ;
    h_b = (int *)malloc((sizeof(int)*m*n)) ;
    h_c = (int *)malloc((sizeof(int)*m*n)) ;
    h_cc = (int *)malloc((sizeof(int)*m*n)) ;

    // random initialize matrix A
    for (int i = 0; i < m; ++i) {
        for (int j = 0; j < n; ++j) {
            h_a[i * n + j] = random()%10;
        }
    }

    // random initialize matrix B
    for (int i = 0; i < n; ++i) {
        for (int j = 0; j < k; ++j) {
            h_b[i * k + j] = random()%10;
        }
    }

    float gpu_elapsed_time_ms, cpu_elapsed_time_ms;

    // some events to count the execution time
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // start to count execution time of GPU version
    // Allocate memory space on the device 
    int *d_a, *d_b, *d_c, *d_bt;
    cudaMalloc(&d_a, sizeof(int)*m*n);
    cudaMalloc(&d_b, sizeof(int)*n*k);
    cudaMalloc(&d_c, sizeof(int)*m*k);
    cudaMalloc(&d_bt, sizeof(int)*m*k);
 
    cudaEventRecord(start, 0);
 
    cudaMemcpy(d_a, h_a, sizeof(int)*m*n, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, sizeof(int)*n*k, cudaMemcpyHostToDevice);

    unsigned int grid_rows = m/ BLOCK_SIZE/size;
    unsigned int grid_cols = k/ BLOCK_SIZE/size;
    dim3 dimGrid(grid_cols, grid_rows);
    //dim3 dimBlock(32, 8);

    dim3 dimGrid2(grid_cols, grid_rows);
    dim3 dimBlock2(BLOCK_SIZE, BLOCK_SIZE); 
 
    //cudaEventRecord(start, 0);
    gpu_matrix_mult<<<dimGrid2, dimBlock2>>>(d_a, d_b, d_c, m, n, k);
    cudaDeviceSynchronize();
    cudaMemcpy(h_c, d_c, sizeof(int)*m*k, cudaMemcpyDeviceToHost);

    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);
 

    cudaEventElapsedTime(&gpu_elapsed_time_ms, start, stop);
    // Transefer results from device to host
    //cudaMemcpy(h_c, d_c, sizeof(int)*m*k, cudaMemcpyDeviceToHost);

    // compute time elapse on GPU computing
    printf("Time elapsed on matrix multiplication of %dx%d . %dx%d on GPU: %f ms.\n\n", m, n, n, k, gpu_elapsed_time_ms);

    // start the CPU version
    cudaEventRecord(start, 0);

    //cpu_matrix_mult(h_a, h_b, h_cc, m, n, k);

    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&cpu_elapsed_time_ms, start, stop);
    //printf("Time elapsed on matrix multiplication of %dx%d . %dx%d on CPU: %f ms.\n\n", m, n, n, k, cpu_elapsed_time_ms);

    // free memory
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    free(h_a);
    free(h_b);
    free(h_c);
    free(h_cc);
    return 0;
}

Time elapsed on matrix multiplication of 8192x8192 . 8192x8192 on GPU: 839.951721 ms.




In [ ]:
%%writefile global.cu
// final version
#include <stdio.h>
#include <stdlib.h>
#include <assert.h>

#define ms 64

#define size 4             // value per thread (square)
#define BLOCK_SIZE 16      // thread per block (square)
#define TILE_WIDTH 64

__global__ void gpu_matrix_mult(float *a,float *b, float *c, int m, int n, int k){ 
    
    __shared__ float A[TILE_WIDTH][TILE_WIDTH];
    __shared__ float B[TILE_WIDTH][TILE_WIDTH];

    int bx = blockIdx.x;
    int by = blockIdx.y;
    int tx = threadIdx.x;
    int ty = threadIdx.y;

    printf("Wadwd");
    float sum[size*size];
    for (int t=0; t<n/TILE_WIDTH; t++)
    {
      for (int j=0; j<size; j++){
          for (int i=0; i<size; i++){
              A[ty+j*BLOCK_SIZE][tx+i*BLOCK_SIZE] = a[by*TILE_WIDTH*n + t*TILE_WIDTH+ty*n+BLOCK_SIZE*n*j+BLOCK_SIZE*i+tx];
              B[ty+j*BLOCK_SIZE][tx+i*BLOCK_SIZE] = b[t*TILE_WIDTH*n + bx*TILE_WIDTH+ty*n+BLOCK_SIZE*n*j+BLOCK_SIZE*i+tx];
          }
      }
      __syncthreads();
      for (int i=0; i<TILE_WIDTH; i++){
          float b_t[size];
          float a_t[size];
          for (int k=0; k<size; k++){
              b_t[k]=B[i][size*tx+k];
              a_t[k]=A[size*ty+k][i];
          }
          for (int q=0; q<size; q++){
              for (int w=0; w<size; w++){
                  sum[size*q+w]+=b_t[w]*a_t[q];
              }
          }
      }
      __syncthreads();
    }
    for (int j=0; j<size; j++){
        for (int i=0; i<size; i++){
            c[by*TILE_WIDTH*n+bx*TILE_WIDTH+size*ty*n+size*tx+i+n*j]=sum[size*j+i];
            //c[by*TILE_WIDTH*n+bx*TILE_WIDTH+j*size*n+ty*n+tx]=sum[size*j+i];
        }
    }
}

void cpu_matrix_mult(float *h_a, float *h_b, float *h_result, int m, int n, int k) {
    for (int i = 0; i < m; ++i) 
    {
        for (int j = 0; j < k; ++j) 
        {
            int tmp = 0.0;
            for (int h = 0; h < n; ++h) 
            {
                tmp += h_a[i * n + h] * h_b[h * k + j];
            }
            h_result[i * k + j] = tmp;
        }
    }
}
int main(int argc, char const *argv[])
{
    int m=ms;
    int n=m;
    int k=m;
 
    srand(3333);

    // allocate memory in host RAM, h_cc is used to store CPU result
    float *h_a, *h_b, *h_c, *h_cc;
    h_a = (float *)malloc((sizeof(float)*m*n)) ;
    h_b = (float *)malloc((sizeof(float)*m*n)) ;
    h_c = (float *)malloc((sizeof(float)*m*n)) ;
    h_cc = (float *)malloc((sizeof(float)*m*n)) ;

    // random initialize matrix A
    for (int i = 0; i < m; ++i) {
        for (int j = 0; j < n; ++j) {
            h_a[i * n + j] = random()%10;
        }
    }

    // random initialize matrix B
    for (int i = 0; i < n; ++i) {
        for (int j = 0; j < k; ++j) {
            h_b[i * k + j] = random()%10;
        }
    }

    float gpu_elapsed_time_ms, cpu_elapsed_time_ms;

    // some events to count the execution time
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // start to count execution time of GPU version
    // Allocate memory space on the device 
    float *d_a, *d_b, *d_c, *d_bt;
    cudaMalloc(&d_a, sizeof(float)*m*n);
    cudaMalloc(&d_b, sizeof(float)*n*k);
    cudaMalloc(&d_c, sizeof(float)*m*k);
    cudaMalloc(&d_bt, sizeof(float)*m*k);
 
    //cudaEventRecord(start, 0);
 
    cudaMemcpy(d_a, h_a, sizeof(float)*m*n, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, sizeof(float)*n*k, cudaMemcpyHostToDevice);

    unsigned int grid_rows = m/ BLOCK_SIZE/size;
    unsigned int grid_cols = k/ BLOCK_SIZE/size;
    dim3 dimGrid(grid_cols, grid_rows);
    //dim3 dimBlock(32, 8);

    dim3 dimGrid2(grid_cols, grid_rows);
    dim3 dimBlock2(BLOCK_SIZE, BLOCK_SIZE); 
 
    cudaEventRecord(start, 0);
    gpu_matrix_mult<<<dimGrid2, dimBlock2>>>(d_a, d_b, d_c, m, n, k);
    cudaDeviceSynchronize();
    cudaEventRecord(stop, 0);
    cudaMemcpy(h_c, d_c, sizeof(float)*m*k, cudaMemcpyDeviceToHost);

    cudaEventSynchronize(stop);
 

    cudaEventElapsedTime(&gpu_elapsed_time_ms, start, stop);
    // Transefer results from device to host
    //cudaMemcpy(h_c, d_c, sizeof(float)*m*k, cudaMemcpyDeviceToHost);

    // compute time elapse on GPU computing
    printf("Time elapsed on matrix multiplication of %dx%d . %dx%d on GPU: %f ms.\n\n", m, n, n, k, gpu_elapsed_time_ms);

    // start the CPU version
    cudaEventRecord(start, 0);

    cpu_matrix_mult(h_a, h_b, h_cc, m, n, k);

    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&cpu_elapsed_time_ms, start, stop);
    //printf("Time elapsed on matrix multiplication of %dx%d . %dx%d on CPU: %f ms.\n\n", m, n, n, k, cpu_elapsed_time_ms);
 int all_ok = 1;
    for (int i = 0; i < m; ++i)
    {
        for (int j = 0; j < k; ++j)
        {
            //printf("\n --%f--, --%f-- \n", h_cc[i*k + j], h_c[i*k + j]);
            //printf("[%d][%d]:%d == [%d][%d]:%d, ", i, j, h_cc[i*k + j], i, j, h_c[i*k + j]);
            if(h_cc[i*k + j] != h_c[i*k + j])
            {
                all_ok = 0;
             
            }
        }
        //printf("\n");
    }

    // roughly compute speedup
    if(all_ok)
    {
        printf("all results are correct!!!, speedup = %f\n", cpu_elapsed_time_ms / gpu_elapsed_time_ms);
    }
    else
    {
        printf("incorrect results\n");
    }

    // free memory
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    free(h_a);
    free(h_b);
    free(h_c);
    free(h_cc);
    return 0;
}

Overwriting global.cu


In [ ]:
!nvcc -o global global.cu

In [ ]:
!./global

Time elapsed on matrix multiplication of 64x64 . 64x64 on GPU: 0.011648 ms.

incorrect results


In [ ]:
%%cu
// paper version
#include <stdio.h>
#include <stdlib.h>
#include <assert.h>

#define ms 1024

#define size 4             // value per thread (square)
#define BLOCK_SIZE 32      // thread per block (square)
#define TILE_WIDTH 64

__global__ void gpu_matrix_mult(float *C,float *A, float *B, int wA, int wB){ 
    
    int bx = blockIdx.x;
    int by = blockIdx.y;
    int tx = threadIdx.x;
    int ty = threadIdx.y;
    int aBegin = wA*BLOCK_SIZE-1;
    int aEnd = aBegin + wA-1;
    int aStep = BLOCK_SIZE;
    int bBegin = BLOCK_SIZE*bx;
    int bStep = BLOCK_SIZE*wB;
    float Csub = 0;

    for (int a = aBegin, b=bBegin; a<=aEnd; a+=aStep, b+=bStep){
        __shared__ float As[BLOCK_SIZE][BLOCK_SIZE];
        __shared__ float Bs[BLOCK_SIZE][BLOCK_SIZE];
        As[ty][tx] = A[a+wA*ty+tx];
        Bs[ty][tx] = B[b+wB*ty+tx];
        __syncthreads();
        for (int k=0; k<BLOCK_SIZE; k++){
            Csub+=As[ty][k]*Bs[k][tx];
        }
        __syncthreads();
    }
    int c = wB*BLOCK_SIZE*by+BLOCK_SIZE*bx;
    C[c+wB*ty+tx]=Csub;
}


int main(int argc, char const *argv[])
{
    int m=ms;
    int n=m;
    int k=m;
 
    srand(3333);

    // allocate memory in host RAM, h_cc is used to store CPU result
    float *h_a, *h_b, *h_c, *h_cc;
    h_a = (float *)malloc((sizeof(float)*m*n)) ;
    h_b = (float *)malloc((sizeof(float)*m*n)) ;
    h_c = (float *)malloc((sizeof(float)*m*n)) ;
    h_cc = (float *)malloc((sizeof(float)*m*n)) ;

    // random initialize matrix A
    for (int i = 0; i < m; ++i) {
        for (int j = 0; j < n; ++j) {
            h_a[i * n + j] = random()%10;
        }
    }

    // random initialize matrix B
    for (int i = 0; i < n; ++i) {
        for (int j = 0; j < k; ++j) {
            h_b[i * k + j] = random()%10;
        }
    }

    float gpu_elapsed_time_ms, cpu_elapsed_time_ms;

    // some events to count the execution time
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // start to count execution time of GPU version
    // Allocate memory space on the device 
    float *d_a, *d_b, *d_c, *d_bt;
    cudaMalloc(&d_a, sizeof(float)*m*n);
    cudaMalloc(&d_b, sizeof(float)*n*k);
    cudaMalloc(&d_c, sizeof(float)*m*k);
    cudaMalloc(&d_bt, sizeof(float)*m*k);
 
    //cudaEventRecord(start, 0);
 
    cudaMemcpy(d_a, h_a, sizeof(float)*m*n, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, sizeof(float)*n*k, cudaMemcpyHostToDevice);

    unsigned int grid_rows = m/ BLOCK_SIZE;
    unsigned int grid_cols = k/ BLOCK_SIZE;
    dim3 dimGrid(grid_cols, grid_rows);
    //dim3 dimBlock(32, 8);

    dim3 dimGrid2(grid_cols, grid_rows);
    dim3 dimBlock2(BLOCK_SIZE, BLOCK_SIZE); 
 
    cudaEventRecord(start, 0);
    gpu_matrix_mult<<<dimGrid2, dimBlock2>>>(d_c, d_a, d_b, m, n);
    cudaDeviceSynchronize();
    cudaEventRecord(stop, 0);
    cudaMemcpy(h_c, d_c, sizeof(float)*m*k, cudaMemcpyDeviceToHost);

    cudaEventSynchronize(stop);
 

    cudaEventElapsedTime(&gpu_elapsed_time_ms, start, stop);
    // Transefer results from device to host
    //cudaMemcpy(h_c, d_c, sizeof(float)*m*k, cudaMemcpyDeviceToHost);

    // compute time elapse on GPU computing
    printf("Time elapsed on matrix multiplication of %dx%d . %dx%d on GPU: %f ms.\n\n", m, n, n, k, gpu_elapsed_time_ms);

    // start the CPU version
    cudaEventRecord(start, 0);

    //cpu_matrix_mult(h_a, h_b, h_cc, m, n, k);

    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&cpu_elapsed_time_ms, start, stop);
    //printf("Time elapsed on matrix multiplication of %dx%d . %dx%d on CPU: %f ms.\n\n", m, n, n, k, cpu_elapsed_time_ms);

    // free memory
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    free(h_a);
    free(h_b);
    free(h_c);
    free(h_cc);
    return 0;
}

UsageError: Cell magic `%%cu` not found.


In [ ]:
%%cu
// fermi
#include <stdio.h>
#include <stdlib.h>
#include <assert.h>

#define ms 8192

#define size 4             // value per thread (square)
#define BLOCK_SIZE 16      // thread per block (square)
#define TILE_WIDTH 64

__global__ void gpu_matrix_mult(float *a,float *b, float *c, int m, int n, int k){ 
    
    __shared__ float A[TILE_WIDTH][TILE_WIDTH/4];
    __shared__ float B[TILE_WIDTH/4][TILE_WIDTH];

    int bx = blockIdx.x;
    int by = blockIdx.y;
    int tx = threadIdx.x;
    int ty = threadIdx.y;
    int row = by * blockDim.y + ty;
    int col = bx * blockDim.x + tx;

    float sum[size*size]={0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0};
    for (int t=0; t<n/16; t++)
    {
      for (int j=0; j<size; j++){
          //A[ty+j*BLOCK_SIZE][tx+i*BLOCK_SIZE] = a[by*TILE_WIDTH*n + t*TILE_WIDTH+ty*n+BLOCK_SIZE*n*j+BLOCK_SIZE*i+tx];
          //B[ty+j*BLOCK_SIZE][tx+i*BLOCK_SIZE] = b[t*TILE_WIDTH*n + bx*TILE_WIDTH+ty*n+BLOCK_SIZE*n*j+BLOCK_SIZE*i+tx];
          B[ty][j*16+tx] = b[bx*TILE_WIDTH+t*16*n+ty*n+tx+j*16];
          A[j*16+ty][tx] = a[by*TILE_WIDTH*n+t*16+j*16*n+ty*n+tx];
          //printf("%f", B[ty][j*16+tx]);
          //printf("%f", A[j*16+ty][tx]);
      }
      __syncthreads();
      for (int i=0; i<TILE_WIDTH/size; i++){
          float b_t[size];
          float a_t[size];
          for (int k=0; k<size; k++){
              b_t[k]=B[i][(TILE_WIDTH/size)*k+tx];
              a_t[k]=A[(TILE_WIDTH/size)*k+ty][i];
          }
          for (int q=0; q<size; q++){
              for (int w=0; w<size; w++){
                  sum[size*q+w]+=b_t[w]*a_t[q];
                  //printf("%f %f %f %f\n", p, sum[size*q+w], b_t[w], a_t[q]);
              }
          }
      }
      __syncthreads();
    }
    for (int j=0; j<size; j++){
        for (int i=0; i<size; i++){
            //c[by*TILE_WIDTH*n+size*ty*n+bx*TILE_WIDTH+size*tx+i+n*j]=sum[size*j+i];
            c[by*TILE_WIDTH*n+bx*TILE_WIDTH+j*size*n+ty*n+tx]=sum[size*j+i];
            //printf("%f ", sum[size*j+i]);
        }
    }
}


int main(int argc, char const *argv[])
{
    int m=ms;
    int n=m;
    int k=m;
 
    srand(3333);

    // allocate memory in host RAM, h_cc is used to store CPU result
    float *h_a, *h_b, *h_c, *h_cc;
    h_a = (float *)malloc((sizeof(float)*m*n)) ;
    h_b = (float *)malloc((sizeof(float)*m*n)) ;
    h_c = (float *)malloc((sizeof(float)*m*n)) ;
    h_cc = (float *)malloc((sizeof(float)*m*n)) ;

    // random initialize matrix A
    for (int i = 0; i < m; ++i) {
        for (int j = 0; j < n; ++j) {
            h_a[i * n + j] = random();
        }
    }

    // random initialize matrix B
    for (int i = 0; i < n; ++i) {
        for (int j = 0; j < k; ++j) {
            h_b[i * k + j] = random();
        }
    }

    float gpu_elapsed_time_ms, cpu_elapsed_time_ms;

    // some events to count the execution time
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // start to count execution time of GPU version
    // Allocate memory space on the device 
    float *d_a, *d_b, *d_c;
    cudaMalloc(&d_a, sizeof(float)*m*n);
    cudaMalloc(&d_b, sizeof(float)*n*k);
    cudaMalloc(&d_c, sizeof(float)*m*k);
 
    //cudaEventRecord(start, 0);
 
    cudaMemcpy(d_a, h_a, sizeof(float)*m*n, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, sizeof(float)*n*k, cudaMemcpyHostToDevice);

    unsigned int grid_rows = m/64;
    unsigned int grid_cols = k/64;
    dim3 dimGrid(grid_cols, grid_rows);

    dim3 dimGrid2(grid_cols, grid_rows);
    dim3 dimBlock2(16, 16); 
 
    cudaEventRecord(start, 0);
    gpu_matrix_mult<<<dimGrid2, dimBlock2>>>(d_a, d_b, d_c, m, n, k);
    cudaDeviceSynchronize();
    cudaEventRecord(stop, 0);
    cudaMemcpy(h_c, d_c, sizeof(float)*m*k, cudaMemcpyDeviceToHost);

    cudaEventSynchronize(stop);
 

    cudaEventElapsedTime(&gpu_elapsed_time_ms, start, stop);
    // Transefer results from device to host
    //cudaMemcpy(h_c, d_c, sizeof(float)*m*k, cudaMemcpyDeviceToHost);

    // compute time elapse on GPU computing
    printf("Time elapsed on matrix multiplication of %dx%d . %dx%d on GPU: %f ms.\n\n", m, n, n, k, gpu_elapsed_time_ms);

    // start the CPU version
    cudaEventRecord(start, 0);

    //cpu_matrix_mult(h_a, h_b, h_cc, m, n, k);

    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&cpu_elapsed_time_ms, start, stop);
    //printf("Time elapsed on matrix multiplication of %dx%d . %dx%d on CPU: %f ms.\n\n", m, n, n, k, cpu_elapsed_time_ms);

    // free memory
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    free(h_a);
    free(h_b);
    free(h_c);
    free(h_cc);
    return 0;
}

Time elapsed on matrix multiplication of 8192x8192 . 8192x8192 on GPU: 354.814789 ms.




In [ ]:
%%writefile fermi.cu
// fermi
#include <stdio.h>
#include <stdlib.h>
#include <assert.h>

#define ms 8192

#define size 4             // value per thread (square)
#define BLOCK_SIZE 16      // thread per block (square)
#define TILE_WIDTH 64

__global__ void gpu_matrix_mult(float *a,float *b, float *c, int m, int n, int k){ 
    
    __shared__ float A[TILE_WIDTH][TILE_WIDTH/4];
    __shared__ float B[TILE_WIDTH/4][TILE_WIDTH];

    int bx = blockIdx.x;
    int by = blockIdx.y;
    int tx = threadIdx.x;
    int ty = threadIdx.y;


    float sum[size*size]={0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0};
    for (int t=0; t<n/16; t++)
    {
      for (int j=0; j<size; j++){
          //A[ty+j*BLOCK_SIZE][tx+i*BLOCK_SIZE] = a[by*TILE_WIDTH*n + t*TILE_WIDTH+ty*n+BLOCK_SIZE*n*j+BLOCK_SIZE*i+tx];
          //B[ty+j*BLOCK_SIZE][tx+i*BLOCK_SIZE] = b[t*TILE_WIDTH*n + bx*TILE_WIDTH+ty*n+BLOCK_SIZE*n*j+BLOCK_SIZE*i+tx];
          B[ty][j*16+tx] = b[bx*TILE_WIDTH+t*16*n+ty*n+tx+j*16];
          A[j*16+ty][tx] = a[by*TILE_WIDTH*n+t*16+j*16*n+ty*n+tx];
          //printf("%f", B[ty][j*16+tx]);
          //printf("%f", A[j*16+ty][tx]);
      }
      __syncthreads();
      for (int i=0; i<TILE_WIDTH/size; i++){
          float b_t[size];
          float a_t[size];
          for (int k=0; k<size; k++){
              b_t[k]=B[i][(TILE_WIDTH/size)*k+tx];
              a_t[k]=A[(TILE_WIDTH/size)*k+ty][i];
          }
          for (int q=0; q<size; q++){
              for (int w=0; w<size; w++){
                  sum[size*q+w]+=b_t[w]*a_t[q];
                  //printf("%f %f %f %f\n", p, sum[size*q+w], b_t[w], a_t[q]);
              }
          }
      }
      __syncthreads();
    }
    for (int j=0; j<size; j++){
        for (int i=0; i<size; i++){
            //c[by*TILE_WIDTH*n+size*ty*n+bx*TILE_WIDTH+size*tx+i+n*j]=sum[size*j+i];
            c[by*TILE_WIDTH*n+bx*TILE_WIDTH+j*size*n+ty*n+tx]=sum[size*j+i];
            //printf("%f ", sum[size*j+i]);
        }
    }
}
void cpu_matrix_mult(float *h_a, float *h_b, float *h_result, int m, int n, int k) {
    for (int i = 0; i < m; ++i) 
    {
        for (int j = 0; j < k; ++j) 
        {
            int tmp = 0.0;
            for (int h = 0; h < n; ++h) 
            {
                tmp += h_a[i * n + h] * h_b[h * k + j];
            }
            h_result[i * k + j] = tmp;
        }
    }
}

int main(int argc, char const *argv[])
{
    int m=ms;
    int n=m;
    int k=m;
 
    srand(3333);

    // allocate memory in host RAM, h_cc is used to store CPU result
    float *h_a, *h_b, *h_c, *h_cc;
    h_a = (float *)malloc((sizeof(float)*m*n)) ;
    h_b = (float *)malloc((sizeof(float)*m*n)) ;
    h_c = (float *)malloc((sizeof(float)*m*n)) ;
    h_cc = (float *)malloc((sizeof(float)*m*n)) ;

    // random initialize matrix A
    for (int i = 0; i < m; ++i) {
        for (int j = 0; j < n; ++j) {
            h_a[i * n + j] = random();
        }
    }

    // random initialize matrix B
    for (int i = 0; i < n; ++i) {
        for (int j = 0; j < k; ++j) {
            h_b[i * k + j] = random();
        }
    }

    float gpu_elapsed_time_ms, cpu_elapsed_time_ms;

    // some events to count the execution time
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // start to count execution time of GPU version
    // Allocate memory space on the device 
    float *d_a, *d_b, *d_c;
    cudaMalloc(&d_a, sizeof(float)*m*n);
    cudaMalloc(&d_b, sizeof(float)*n*k);
    cudaMalloc(&d_c, sizeof(float)*m*k);
 
    //cudaEventRecord(start, 0);
 
    cudaMemcpy(d_a, h_a, sizeof(float)*m*n, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, sizeof(float)*n*k, cudaMemcpyHostToDevice);

    unsigned int grid_rows = m/64;
    unsigned int grid_cols = k/64;
    dim3 dimGrid(grid_cols, grid_rows);

    dim3 dimGrid2(grid_cols, grid_rows);
    dim3 dimBlock2(16, 16); 
 
    cudaEventRecord(start, 0);
    gpu_matrix_mult<<<dimGrid2, dimBlock2>>>(d_a, d_b, d_c, m, n, k);
    cudaDeviceSynchronize();
    cudaEventRecord(stop, 0);
    cudaMemcpy(h_c, d_c, sizeof(float)*m*k, cudaMemcpyDeviceToHost);

    cudaEventSynchronize(stop);
 

    cudaEventElapsedTime(&gpu_elapsed_time_ms, start, stop);
    // Transefer results from device to host
    //cudaMemcpy(h_c, d_c, sizeof(float)*m*k, cudaMemcpyDeviceToHost);

    // compute time elapse on GPU computing
    printf("Time elapsed on matrix multiplication of %dx%d . %dx%d on GPU: %f ms.\n\n", m, n, n, k, gpu_elapsed_time_ms);

    // start the CPU version
    cudaEventRecord(start, 0);

    //cpu_matrix_mult(h_a, h_b, h_cc, m, n, k);

    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&cpu_elapsed_time_ms, start, stop);
    //printf("Time elapsed on matrix multiplication of %dx%d . %dx%d on CPU: %f ms.\n\n", m, n, n, k, cpu_elapsed_time_ms);
    int all_ok = 1;
    for (int i = 0; i < m; ++i)
    {
        for (int j = 0; j < k; ++j)
        {
            //printf("\n --%f--, --%f-- \n", h_cc[i*k + j], h_c[i*k + j]);
            //printf("[%d][%d]:%d == [%d][%d]:%d, ", i, j, h_cc[i*k + j], i, j, h_c[i*k + j]);
            if(h_cc[i*k + j] != h_c[i*k + j])
            {
                all_ok = 0;
            }
        }
        //printf("\n");
    }

    // roughly compute speedup
    if(all_ok)
    {
        printf("all results are correct!!!, speedup = %f\n", cpu_elapsed_time_ms / gpu_elapsed_time_ms);
    }
    else
    {
        printf("incorrect results\n");
    }

    // free memory
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    free(h_a);
    free(h_b);
    free(h_c);
    free(h_cc);
    return 0;
}

Overwriting fermi.cu


In [ ]:
!nvcc -o fermi fermi.cu

In [ ]:
!./fermi

Time elapsed on matrix multiplication of 8192x8192 . 8192x8192 on GPU: 365.946350 ms.

incorrect results


In [ ]:
!/usr/local/cuda-10.0/NsightCompute-1.0/nv-nsight-cu-cli ./fermi

In [ ]:
import torch
print('__CUDA Device Name:',torch.cuda.get_device_name(0))

__CUDA Device Name: Tesla K80


In [ ]:
%%writefile 16x8_128x128_v3.cu
// fermi
#include <stdio.h>
#include <stdlib.h>
#include <assert.h>
#define check_with_cpu 0
#define ms 8192
#define bs_x 16
#define bs_y 8
#define TILE_WIDTH_Y 128
#define TILE_WIDTH_X 128

__global__ void gpu_matrix_mult(float *a,float *b, float *c, int m, int n, int k){ 
    
    __shared__ float A[128][16];
    __shared__ float B[16][128];

    int bx = blockIdx.x;
    int by = blockIdx.y;
    int tx = threadIdx.x;
    int ty = threadIdx.y;

    float sum[128]={0};
    for (int t=0; t<n/16; t++)
    {
      for (int i=0; i<2; i++){
          for (int j=0; j<8; j++){
              B[ty+8*i][tx+16*j] = b[bx*TILE_WIDTH_X+t*16*n+(ty+8*i)*n+tx+16*j];
          }
      }
      for (int j=0; j<16; j++){
          A[ty+j*8][tx] = a[by*TILE_WIDTH_Y*n+t*16+(ty+j*8)*n+tx];
      }
      __syncthreads();
      for (int i=0; i<16; i++){
          float b_t[8];
          float a_t[16];
          for (int k=0; k<8; k++){
              b_t[k]=B[i][16*k+tx];
          }
          for (int k=0; k<16; k++){
              a_t[k]=A[8*k+ty][i];
          }
          for (int q=0; q<16; q++){
              for (int w=0; w<8; w++){
                  sum[8*q+w]+=b_t[w]*a_t[q];
                  //printf("%f %f", b_t[w], a_t[q]);
              }
          }
      }
      __syncthreads();
    }
    for (int j=0; j<16; j++){
        for (int i=0; i<8; i++){
            c[by*TILE_WIDTH_Y*n+bx*TILE_WIDTH_X+j*bs_y*n+bs_x*i+ty*n+tx]=sum[8*j+i];
            //printf("%f ", sum[8*j+i]);
        }
    }
}
void cpu_matrix_mult(float *h_a, float *h_b, float *h_result, int m, int n, int k) {
    for (int i = 0; i < m; ++i) 
    {
        for (int j = 0; j < k; ++j) 
        {
            float tmp = 0.0;
            for (int h = 0; h < n; ++h) 
            {
                tmp += h_a[i * n + h] * h_b[h * k + j];
            }
            h_result[i * k + j] = tmp;
        }
    }
}

int main(int argc, char const *argv[])
{
   
    int m=ms;
    int n=m;
    int k=m;
 
    srand(3333);

    // allocate memory in host RAM, h_cc is used to store CPU result
    float *h_a, *h_b, *h_c, *h_cc;
    h_a = (float *)malloc((sizeof(float)*m*n)) ;
    h_b = (float *)malloc((sizeof(float)*m*n)) ;
    h_c = (float *)malloc((sizeof(float)*m*n)) ;
    h_cc = (float *)malloc((sizeof(float)*m*n)) ;

    // random initialize matrix A
    for (int i = 0; i < m; ++i) {
        for (int j = 0; j < n; ++j) {
            h_a[i * n + j] =random()%5;
        }
    }

    // random initialize matrix B
    for (int i = 0; i < n; ++i) {
        for (int j = 0; j < k; ++j) {
            h_b[i * k + j] = random()%5;
        }
    }

    float gpu_elapsed_time_ms, cpu_elapsed_time_ms;

    // some events to count the execution time
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // start to count execution time of GPU version
    // Allocate memory space on the device 
    float *d_a, *d_b, *d_c;
    cudaMalloc(&d_a, sizeof(float)*m*n);
    cudaMalloc(&d_b, sizeof(float)*n*k);
    cudaMalloc(&d_c, sizeof(float)*m*k);
 
    cudaEventRecord(start, 0);
 
    cudaMemcpy(d_a, h_a, sizeof(float)*m*n, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, sizeof(float)*n*k, cudaMemcpyHostToDevice);

    unsigned int grid_rows = m/TILE_WIDTH_Y;
    unsigned int grid_cols = k/TILE_WIDTH_X;
  

    dim3 dimGrid2(grid_cols, grid_rows);
    dim3 dimBlock2(16, 8); 
 
    cudaEventRecord(start, 0);
    gpu_matrix_mult<<<dimGrid2, dimBlock2>>>(d_a, d_b, d_c, m, n, k);
    cudaDeviceSynchronize();
    cudaEventRecord(stop, 0);
    cudaMemcpy(h_c, d_c, sizeof(float)*m*k, cudaMemcpyDeviceToHost);

    cudaEventSynchronize(stop);
 

    cudaEventElapsedTime(&gpu_elapsed_time_ms, start, stop);
    // Transefer results from device to host
    //cudaMemcpy(h_c, d_c, sizeof(float)*m*k, cudaMemcpyDeviceToHost);

    // compute time elapse on GPU computing
    printf("Time elapsed on matrix multiplication of %dx%d . %dx%d on GPU: %f ms.\n\n", m, n, n, k, gpu_elapsed_time_ms);

    // start the CPU version
    cudaEventRecord(start, 0);
  
    if (check_with_cpu==1){
    cpu_matrix_mult(h_a, h_b, h_cc, m, n, k);

    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&cpu_elapsed_time_ms, start, stop);
    //printf("Time elapsed on matrix multiplication of %dx%d . %dx%d on CPU: %f ms.\n\n", m, n, n, k, cpu_elapsed_time_ms);
    int all_ok = 1;
    for (int i = 0; i < m; ++i)
    {
        for (int j = 0; j < k; ++j)
        {
            //printf("\n --%f--, --%f-- \n", h_cc[i*k + j], h_c[i*k + j]);
            //printf("[%f][%f]:%f == [%f][%f]:%f, ", i, j, h_cc[i*k + j], i, j, h_c[i*k + j]);
            if(h_cc[i*k + j] != h_c[i*k + j])
            {
                all_ok = 0;
            }
        }
        //printf("\n");
    }

    // roughly compute speedup
    if(all_ok)
    {
        printf("all results are correct!!!, speedup = %f\n", cpu_elapsed_time_ms / gpu_elapsed_time_ms);
    }
    else
    {
        printf("incorrect results\n");
    }
    }
    // free memory
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    free(h_a);
    free(h_b);
    free(h_c);
    free(h_cc);
    
    return 0;
}

Writing 16x8_128x128_v3.cu


In [ ]:
!nvcc -o 16x8_128x128_v3 16x8_128x128_v3.cu 

In [ ]:
!nvprof ./16x8_128x128_v3

==197== NVPROF is profiling process 197, command: ./16x8_128x128_v3
==197== Warning: Auto boost enabled on device 0. Profiling results may be inconsistent.
Time elapsed on matrix multiplication of 8192x8192 . 8192x8192 on GPU: 0.012320 ms.

==197== Profiling application: ./16x8_128x128_v3
==197== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   68.10%  154.82ms         1  154.82ms  154.82ms  154.82ms  [CUDA memcpy DtoH]
                   31.90%  72.529ms         2  36.264ms  36.229ms  36.299ms  [CUDA memcpy HtoD]
      API calls:   45.87%  228.58ms         3  76.193ms  36.324ms  155.92ms  cudaMemcpy
                   39.85%  198.54ms         2  99.271ms  1.8730us  198.54ms  cudaEventCreate
                   13.81%  68.788ms         3  22.929ms  456.32us  34.258ms  cudaFree
                    0.29%  1.4576ms         3  485.86us  445.59us  529.40us  cudaMalloc
                    0.09%  466.05us         1  466.05us